# Windows에서 VLM 사용하기

Windows 환경에서는 MLX가 지원되지 않으므로, Hugging Face Transformers를 사용하여 VLM을 구현합니다.

## 지원되는 VLM 모델들

- **LLaVA (Large Language and Vision Assistant)**: 텍스트와 이미지를 함께 처리
- **BLIP-2**: 이미지 캡셔닝과 VQA (Visual Question Answering)
- **InstructBLIP**: 지시사항 기반 멀티모달 모델
- **MiniGPT-4**: GPT-4와 유사한 멀티모달 능력

## 설치 요구사항

```bash
uv add transformers torch torchvision pillow
```

### 필요한 라이브러리 임포트

In [ ]:
import torch
from transformers import (
    BlipProcessor, 
    BlipForConditionalGeneration,
    AutoProcessor,
    LlavaForConditionalGeneration
)
from PIL import Image
import requests
from io import BytesIO

### BLIP-2를 사용한 이미지 캡셔닝 예제

BLIP-2는 이미지를 보고 설명을 생성하는 강력한 VLM입니다.

In [ ]:
# BLIP-2 모델과 프로세서 로드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# 샘플 이미지 로드 (인터넷에서)
url = "https://images.unsplash.com/photo-1518717758536-85ae29035b6d?w=400"
response = requests.get(url)
image = Image.open(BytesIO(response.content))

# 이미지 처리 및 캡션 생성
inputs = processor(image, return_tensors="pt")
out = model.generate(**inputs, max_length=50)
caption = processor.decode(out[0], skip_special_tokens=True)

print(f"이미지 설명: {caption}")
image.show()  # 이미지 표시

### LLaVA를 사용한 Visual Question Answering

LLaVA는 이미지에 대한 질문에 답할 수 있는 고급 VLM입니다.

In [ ]:
# LLaVA 모델 로드 (작은 버전 사용)
model_id = "llava-hf/llava-1.5-7b-hf"

# GPU가 있으면 사용, 없으면 CPU 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)

print(f"모델이 {device}에서 실행됩니다.")

In [ ]:
# 이미지와 질문 준비
url = "https://images.unsplash.com/photo-1551963831-b3b1ca40c98e?w=400"
response = requests.get(url)
image = Image.open(BytesIO(response.content))

# 질문 설정
question = "이 이미지에서 무엇을 볼 수 있나요? 자세히 설명해주세요."
prompt = f"USER: <image>\n{question}\nASSISTANT:"

# 입력 처리
inputs = processor(prompt, image, return_tensors="pt").to(device, torch_dtype)

# 답변 생성
generate_ids = model.generate(
    **inputs, 
    max_new_tokens=200, 
    do_sample=True, 
    temperature=0.7
)

# 결과 디코딩
response = processor.batch_decode(
    generate_ids[:, inputs['input_ids'].shape[1]:], 
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=False
)[0]

print(f"질문: {question}")
print(f"답변: {response}")
image.show()

### PDF 문서 처리를 위한 VLM 활용

PDF 페이지를 이미지로 변환하고 VLM으로 분석하는 예제입니다.

In [ ]:
# PDF 처리를 위한 추가 라이브러리 (필요시 설치)
# uv add pdf2image

def analyze_pdf_page_with_vlm(pdf_path, page_number=0):
    """
    PDF 페이지를 VLM으로 분석하는 함수
    """
    try:
        from pdf2image import convert_from_path
        
        # PDF를 이미지로 변환
        pages = convert_from_path(pdf_path)
        page_image = pages[page_number]
        
        # VLM으로 페이지 분석
        question = "이 문서 페이지의 내용을 요약하고, 주요 정보를 추출해주세요. 표, 그래프, 이미지가 있다면 설명해주세요."
        prompt = f"USER: <image>\n{question}\nASSISTANT:"
        
        inputs = processor(prompt, page_image, return_tensors="pt").to(device, torch_dtype)
        
        generate_ids = model.generate(
            **inputs, 
            max_new_tokens=300, 
            do_sample=True, 
            temperature=0.7
        )
        
        response = processor.batch_decode(
            generate_ids[:, inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )[0]
        
        return response, page_image
        
    except ImportError:
        print("pdf2image 라이브러리가 필요합니다. 'uv add pdf2image'로 설치해주세요.")
        return None, None

# 사용 예제 (PDF 파일이 있는 경우)
# analysis, image = analyze_pdf_page_with_vlm("sample.pdf", 0)
# if analysis:
#     print(f"페이지 분석 결과: {analysis}")
#     image.show()

## RAG 시스템과의 통합

VLM을 RAG 파이프라인에 통합하는 방법:

In [ ]:
class VLMRAGProcessor:
    """
    VLM을 활용한 RAG 처리기
    """
    
    def __init__(self, vlm_model, vlm_processor):
        self.vlm_model = vlm_model
        self.vlm_processor = vlm_processor
        self.device = next(vlm_model.parameters()).device
        
    def extract_visual_content(self, image, context_question=None):
        """
        이미지에서 텍스트 정보를 추출
        """
        if context_question is None:
            context_question = "이 이미지의 모든 텍스트, 표, 차트, 그래프 내용을 자세히 설명해주세요. 구조와 데이터를 포함해서 설명해주세요."
        
        prompt = f"USER: <image>\n{context_question}\nASSISTANT:"
        
        inputs = self.vlm_processor(prompt, image, return_tensors="pt").to(self.device)
        
        generate_ids = self.vlm_model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.3  # 더 일관된 결과를 위해 낮은 온도
        )
        
        response = self.vlm_processor.batch_decode(
            generate_ids[:, inputs['input_ids'].shape[1]:],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]
        
        return response
    
    def answer_visual_question(self, image, question):
        """
        이미지 기반 질문 답변
        """
        prompt = f"USER: <image>\n{question}\nASSISTANT:"
        
        inputs = self.vlm_processor(prompt, image, return_tensors="pt").to(self.device)
        
        generate_ids = self.vlm_model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7
        )
        
        response = self.vlm_processor.batch_decode(
            generate_ids[:, inputs['input_ids'].shape[1]:],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]
        
        return response

# VLM RAG 프로세서 초기화
vlm_rag = VLMRAGProcessor(model, processor)

print("VLM RAG 프로세서가 준비되었습니다.")

## 성능 최적화 팁

### 1. GPU 메모리 관리
```python
# GPU 메모리 정리
torch.cuda.empty_cache()
```

### 2. 배치 처리
여러 이미지를 한 번에 처리하여 효율성 향상

### 3. 모델 양자화
```python
# 4비트 양자화로 메모리 사용량 감소
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
```

## 결론

Windows에서는 MLX 대신 Hugging Face Transformers를 사용하여 VLM을 구현할 수 있습니다. 주요 장점:

1. **광범위한 모델 지원**: LLaVA, BLIP-2, InstructBLIP 등
2. **GPU 가속**: CUDA 지원으로 빠른 추론
3. **유연한 통합**: 기존 RAG 시스템과 쉬운 통합
4. **활발한 커뮤니티**: 지속적인 업데이트와 지원

이 방법으로 Windows에서도 강력한 VLM 기반 RAG 시스템을 구축할 수 있습니다.